In [ ]:
# ==============================================================================
# CELL 1: INSTALL DEPENDENCIES
# ==============================================================================
!pip install -q -U langchain-community langchain-huggingface sentence-transformers faiss-cpu langchain openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
# ==============================================================================
# CELL 2: IMPORTS & CONFIGURATION
# ==============================================================================
import os
import json
import pandas as pd
from google.colab import drive
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. CONFIGURATION PATHS -- UPDATE THESE IF NEEDED
BASE_DIR = "/content/drive/MyDrive"
INPUT_TEXT_DIR = os.path.join("/content/drive/MyDrive/FINAL_FOLDER/Text Extraction/text_extracted")     # Folder with .txt files
OUTPUT_RESULTS_DIR = os.path.join("/content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new") # Folder to save results

# Path to your uploaded metadata CSV
# Update this path to where your metadata file actually lives in Drive or Colab
METADATA_FILE_PATH = "/content/drive/MyDrive/FINAL_FOLDER/Text Extraction/metadata_new_final_v1.csv"

# Create output directory
os.makedirs(OUTPUT_RESULTS_DIR, exist_ok=True)

print(f"📂 Input Directory: {INPUT_TEXT_DIR}")
print(f"📂 Output Directory: {OUTPUT_RESULTS_DIR}")
print(f"📄 Metadata File: {METADATA_FILE_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Input Directory: /content/drive/MyDrive/FINAL_FOLDER/Text Extraction/text_extracted
📂 Output Directory: /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new
📄 Metadata File: /content/drive/MyDrive/FINAL_FOLDER/Text Extraction/metadata_new_final_v1.csv


In [ ]:
# ==============================================================================
# CELL 3: LOAD METADATA
# ==============================================================================
def load_metadata_map(csv_path):
    """
    Loads the CSV and creates a dictionary for fast lookups by filename.
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"❌ Metadata file not found at: {csv_path}")

    # Read CSV
    df = pd.read_csv(csv_path)

    # Create a dictionary where key = filename, value = row as dict
    meta_map = {}
    for _, row in df.iterrows():
        # Clean the filename to ensure it matches (strip whitespace)
        fname = str(row['filename']).strip()
        meta_map[fname] = {
            "title": row['title'],
            "doi": row['doi'],
            "year": row['year'],
            "publication": row.get('publication', 'N/A')
        }

    print(f"✅ Loaded metadata for {len(meta_map)} files.")
    return meta_map

# Load the map immediately
try:
    METADATA_MAP = load_metadata_map(METADATA_FILE_PATH)
except Exception as e:
    print(f"⚠️ Warning: Could not load metadata ({e}). Proceeding without it.")
    METADATA_MAP = {}

✅ Loaded metadata for 3746 files.


In [ ]:
# ==============================================================================
# CELL 4: ROBUST TOKEN CHUNKING LOGIC
# ==============================================================================
def split_text_by_tokens(text, filename):
    """
    Splits text into fixed-size chunks based on tokens/characters.
    Looks up metadata from the global METADATA_MAP.
    """
    # 1. Get Metadata for this file
    file_meta = METADATA_MAP.get(filename.strip(), {})

    # Defaults if metadata is missing
    doc_title = file_meta.get("title", filename)
    doc_doi = file_meta.get("doi", "Unknown DOI")
    doc_year = file_meta.get("year", "Unknown Year")

    # 2. Configure the Splitter
    # chunk_size=1000 characters is roughly 250-300 words.
    # We use a large overlap (200) to ensure context isn't lost at the cut points.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,      # Large chunk size (~1000 tokens)
        chunk_overlap=500,    # Generous overlap to preserve context
        length_function=len,
        separators=["\n\n", "\n", " ", ""] # Try to split by paragraphs first
    )

    # 3. Create Documents
    raw_chunks = text_splitter.split_text(text)
    docs = []

    for i, chunk_content in enumerate(raw_chunks):
        doc = Document(
            page_content=chunk_content,
            metadata={
                "source": filename,
                "chunk_id": i,        # specific ID for this chunk
                "title": doc_title,   # <-- From CSV
                "doi": doc_doi,       # <-- From CSV
                "year": doc_year      # <-- From CSV
            }
        )
        docs.append(doc)

    return docs

In [ ]:
# ==============================================================================
# CELL 5: BUILD VECTOR DATABASE (One-Time Process)
# ==============================================================================
print("⏳ Initializing Embedding Model...")
# Using all-MiniLM-L6-v2. It has a context limit of 512 tokens.
# Note: If your chunks are 4000 chars (~1000 tokens), this model will truncate
# the end of the chunk during embedding.
# To fully capture 1000+ tokens, consider using 'BAAI/bge-large-en-v1.5' or similar.
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("⏳ Processing files and building index...")
all_chunks = []

if os.path.exists(INPUT_TEXT_DIR):
    files = [f for f in os.listdir(INPUT_TEXT_DIR) if f.endswith(".txt")]

    for filename in files:
        path = os.path.join(INPUT_TEXT_DIR, filename)
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            raw_text = f.read()

        # Split and enrich with metadata
        file_chunks = split_text_by_tokens(raw_text, filename)
        all_chunks.extend(file_chunks)

    print(f"✅ Processed {len(files)} files into {len(all_chunks)} chunks.")

    print("⏳ Building FAISS Index...")
    vector_db = FAISS.from_documents(all_chunks, embedding_model)
    print("✅ Vector Database Ready!")
else:
    print(f"❌ Error: Input directory '{INPUT_TEXT_DIR}' does not exist.")

⏳ Initializing Embedding Model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

⏳ Processing files and building index...
✅ Processed 3746 files into 58807 chunks.
⏳ Building FAISS Index...
✅ Vector Database Ready!


In [ ]:
# ==============================================================================
# CELL 6: QUERY & SAVE LOGIC (Updated for Top 10 Papers)
# ==============================================================================
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def process_and_save_query(user_query):
    print(f"\n🔎 Processing: '{user_query}'")

    # 1. Retrieve (Broad Search - Get top 50 candidates)
    initial_docs = vector_db.similarity_search(user_query, k=50)

    # 2. Rerank (Precision Sorting)
    pairs = [[user_query, doc.page_content] for doc in initial_docs]
    scores = reranker.predict(pairs)

    # Sort by score (High to Low)
    ranked_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

    # 3. Output 1: Top 10 Chunks (Specific Content)
    top_chunks_data = []
    for doc, score in ranked_docs[:10]:
        top_chunks_data.append({
            "score": float(score),
            "text": doc.page_content,
            "chunk_id": doc.metadata.get("chunk_id"),
            "paper_title": doc.metadata.get("title"),
            "doi": doc.metadata.get("doi"),
            "year": doc.metadata.get("year"),
            "filename": doc.metadata.get("source")
        })

    # 4. Output 2: Top 10 Unique Papers (Sources)
    top_papers_data = []
    seen_titles = set()

    for doc, score in ranked_docs:
        title = doc.metadata.get("title", "Unknown")
        # Only add if we haven't seen this paper yet
        if title not in seen_titles:
            top_papers_data.append({
                "rank": len(top_papers_data) + 1,
                "title": title,
                "doi": doc.metadata.get("doi"),
                "year": doc.metadata.get("year"),
                "best_chunk_score": float(score),
                "filename": doc.metadata.get("source")
            })
            seen_titles.add(title)

        # Stop once we have 10 papers
        if len(top_papers_data) >= 10:
            break

    # 5. Save to Drive
    safe_name = "".join([c if c.isalnum() else "_" for c in user_query])[:50]

    chunks_file = os.path.join(OUTPUT_RESULTS_DIR, f"CHUNKS_{safe_name}.json")
    papers_file = os.path.join(OUTPUT_RESULTS_DIR, f"PAPERS_{safe_name}.json")

    with open(chunks_file, 'w') as f:
        json.dump(top_chunks_data, f, indent=4)

    with open(papers_file, 'w') as f:
        json.dump(top_papers_data, f, indent=4)

    print(f"✅ Results saved to:\n   - {chunks_file}\n   - {papers_file}")
    return top_chunks_data, top_papers_data

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: How have the rates of liver transplantation for HCC changed over the last 10 years, considering the implementation of 6 month waiting period?

🔎 Processing: 'How have the rates of liver transplantation for HCC changed over the last 10 years, considering the implementation of 6 month waiting period?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_How_have_the_rates_of_liver_transplantation_for_HC.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_How_have_the_rates_of_liver_transplantation_for_HC.json

TOP 10 RELEVANT PAPERS FOR: 'How have the rates of liver transplantation for HCC changed over the last 10 years, considering the implementation of 6 month waiting period?'
#1 [Score: 1.8211]
   Title: Management of patients with liver diseases
   DOI:   10/1186/s12916-018-1110-y
--------------------------------------------------
#2 [Score: 1.7927]
   Title: CA S E R E POR T Open 

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: What are factors associated with increased use of living donor liver transplantation across the country?

🔎 Processing: 'What are factors associated with increased use of living donor liver transplantation across the country?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_What_are_factors_associated_with_increased_use_of_.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_What_are_factors_associated_with_increased_use_of_.json

TOP 10 RELEVANT PAPERS FOR: 'What are factors associated with increased use of living donor liver transplantation across the country?'
#1 [Score: 1.6593]
   Title: Importance of Cardiovascular Function in
   DOI:   10/4097/kjae/2018/71/2/85
--------------------------------------------------
#2 [Score: 0.9329]
   Title: Outcomes of transplantation of livers
   DOI:   10/1136/bmjopen-2013-003287
--------------------------------------------------
#3 [Scor

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: How has the evidence changed regarding acceptable macro/microsteatosis in liver grafts?

🔎 Processing: 'How has the evidence changed regarding acceptable macro/microsteatosis in liver grafts?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_How_has_the_evidence_changed_regarding_acceptable_.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_How_has_the_evidence_changed_regarding_acceptable_.json

TOP 10 RELEVANT PAPERS FOR: 'How has the evidence changed regarding acceptable macro/microsteatosis in liver grafts?'
#1 [Score: -1.0082]
   Title: R E S EAR CH Open Access
   DOI:   10/1186/1477-5751-12-18
--------------------------------------------------
#2 [Score: -2.5786]
   Title: J Clin Pathol 1985;38:1366-1373
   DOI:   10/1136/jcp/38/12/1366
--------------------------------------------------
#3 [Score: -2.7999]
   Title: https://doi.org/10.1038/s41586-018-0047-9
   DOI:   10/1

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: How has machine perfusion changed risk thresholds associated with steatotic donor livers?

🔎 Processing: 'How has machine perfusion changed risk thresholds associated with steatotic donor livers?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_How_has_machine_perfusion_changed_risk_thresholds_.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_How_has_machine_perfusion_changed_risk_thresholds_.json

TOP 10 RELEVANT PAPERS FOR: 'How has machine perfusion changed risk thresholds associated with steatotic donor livers?'
#1 [Score: 1.6912]
   Title: published: 12 May 2020
   DOI:   10/3389/fmed/2020/00135
--------------------------------------------------
#2 [Score: 1.1413]
   Title: Improvement of steatotic rat liver function
   DOI:   10/1371/journal/pone/0232886
--------------------------------------------------
#3 [Score: 0.6005]
   Title: https://doi.org/10.1038/s41586-018-00

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: What is the impact of macro/microsteatosis on post-transplant graft survival taking into account donor and other factors?

🔎 Processing: 'What is the impact of macro/microsteatosis on post-transplant graft survival taking into account donor and other factors?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_What_is_the_impact_of_macro_microsteatosis_on_post.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_What_is_the_impact_of_macro_microsteatosis_on_post.json

TOP 10 RELEVANT PAPERS FOR: 'What is the impact of macro/microsteatosis on post-transplant graft survival taking into account donor and other factors?'
#1 [Score: -0.0826]
   Title: Effects of Hyperuricemia on Renal Function of Renal
   DOI:   10/1371/journal/pone/0039457
--------------------------------------------------
#2 [Score: -0.1908]
   Title: University of Groningen
   DOI:   10/1016/j/trre/2018/06/001
-------

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: What are the current research/knowledge gaps in standardizing perfusion protocols?

🔎 Processing: 'What are the current research/knowledge gaps in standardizing perfusion protocols?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_What_are_the_current_research_knowledge_gaps_in_st.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_What_are_the_current_research_knowledge_gaps_in_st.json

TOP 10 RELEVANT PAPERS FOR: 'What are the current research/knowledge gaps in standardizing perfusion protocols?'
#1 [Score: -0.5246]
   Title: HYPO TH E SI S Open Access
   DOI:   10/1186/s40779-020-00250-y
--------------------------------------------------
#2 [Score: -0.7471]
   Title: published: 12 May 2020
   DOI:   10/3389/fimmu/2020/00631
--------------------------------------------------
#3 [Score: -0.8773]
   Title: Hydroxyethyl starch for perioperative goal￾directed fluid therapy in 2020

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: Which technologies show the strongest publication/citation increases in the last five years indicating emerging interests/trends/consensus (outside of the expected like AI or by center or region)?

🔎 Processing: 'Which technologies show the strongest publication/citation increases in the last five years indicating emerging interests/trends/consensus (outside of the expected like AI or by center or region)?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_Which_technologies_show_the_strongest_publication_.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_Which_technologies_show_the_strongest_publication_.json

TOP 10 RELEVANT PAPERS FOR: 'Which technologies show the strongest publication/citation increases in the last five years indicating emerging interests/trends/consensus (outside of the expected like AI or by center or region)?'
#1 [Score: -1.9888]
   Title: NAR RA TIV E R 

In [ ]:
# ==============================================================================
# CELL 7: RUN IT
# ==============================================================================
query = input("Enter your research question: ")
if query:
    chunks, papers = process_and_save_query(query)

    print("\n" + "="*50)
    print(f"TOP 10 RELEVANT PAPERS FOR: '{query}'")
    print("="*50)

    for p in papers:
        print(f"#{p['rank']} [Score: {p['best_chunk_score']:.4f}]")
        print(f"   Title: {p['title']}")
        print(f"   DOI:   {p['doi']}")
        print("-" * 50)

Enter your research question: How have innovations / impact of policies spread through the literature over time?

🔎 Processing: 'How have innovations / impact of policies spread through the literature over time?'
✅ Results saved to:
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/CHUNKS_How_have_innovations___impact_of_policies_spread_t.json
   - /content/drive/MyDrive/FINAL_FOLDER/Maitri/retrieval_results_new/PAPERS_How_have_innovations___impact_of_policies_spread_t.json

TOP 10 RELEVANT PAPERS FOR: 'How have innovations / impact of policies spread through the literature over time?'
#1 [Score: -4.0225]
   Title: R E S EAR CH Open Access
   DOI:   10/1186/s41124-018-0036-5
--------------------------------------------------
#2 [Score: -6.3437]
   Title: Impact of presumed consent for organ donation on donation
   DOI:   10/1136/bmj/a3162
--------------------------------------------------
#3 [Score: -7.6723]
   Title: It has been claimed and demonstrated that many (